In [2]:
# Read from Hop's Github to pull bridge tokens and addresses for Dune's spell table
# Right now, we manually run this and copy and paste to Dune's spellbook
# it would be cool to automate this process some day...

import requests
import re
import json
import pandas as pd
import sys 
sys.path.append('../')
from dune_exports import *
sys.path.pop()

gh_api = 'https://api.github.com/repos/OWNER/REPO/contents/PATH'
#sourcefile = 'https://github.com/hop-protocol/hop/blob/develop/packages/core/src/addresses/mainnet.ts'
owner_api = 'hop-protocol'
repo_api='hop'
path_api = 'packages/core/src/addresses/'
file_names = ['mainnet.ts','types.ts']

In [3]:
for a_fn in file_names:
        # Code from StackOverflow - can't find the link now
        rep = {'OWNER': owner_api, 'REPO': repo_api, 'PATH':path_api + a_fn} # define desired replacements here
        ## use these three lines to do the replacement
        rep = dict((re.escape(k), v) for k, v in rep.items()) 
        ## Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
        rep_pattern = re.compile("|".join(rep.keys()))
        req_api = rep_pattern.sub(lambda m: rep[re.escape(m.group(0))], gh_api)
        # print(req_api)

        inf = requests.get(req_api)
        dl_url = inf.json()['download_url']
        print(dl_url)

        r = requests.get(dl_url)
        f = open(a_fn, 'wb')
        for chunk in r.iter_content(chunk_size=512 * 1024): 
                if chunk:
                        f.write(chunk)
        f.close()


# json = json.dumps(content)
# print(json)

https://raw.githubusercontent.com/hop-protocol/hop/develop/packages/core/src/addresses/mainnet.ts
https://raw.githubusercontent.com/hop-protocol/hop/develop/packages/core/src/addresses/types.ts


In [4]:
# read jsons to intermediates folder
! node hop_transpile.js

zsh:1: command not found: node


In [5]:
bridges = pd.read_json("intermediates/hop_bridges.txt")
bridges.reset_index(inplace=True)
bridges_unpivot = pd.melt(bridges, id_vars='index')
bridges_unpivot.reset_index(inplace=True, drop=True)

# expand data in json to columns
json_data = bridges_unpivot['value'].values
normalized_df = pd.json_normalize(json_data)
bridges = bridges_unpivot.join(normalized_df)
bridges = bridges[~bridges['bridgeDeployedBlockNumber'].isna()]
# drop original json column
bridges = bridges.drop('value', axis=1)
#rename columns
bridges = bridges.rename(columns={
        'index':'blockchain',
        'variable':'token_symbol'
})

int_cols = ['bridgeDeployedBlockNumber']

to_dune_sql_format(bridges, 'hop', int_cols)



In [6]:
# #write sql file
# bridges.to_csv('outputs/hop_to_sql.txt', sep='\t', index=False)
# #write cols
# text_file = open('outputs/hop_dune_column_names.txt', "w")
# n = text_file.write(col_names)
# text_file.close()